# Using Selenium for scraping of infinity pages

## Step 1
* conda install selenium
* Install the geckodriver to run firefox in the background

https://github.com/mozilla/geckodriver/releases

Download the binary for your system and place the binary in the same folder as this Jupyter notebook

![images/assets.jpg](images/assets.jpg)

## Step 2

In [ ]:
#infinityscroll - via Geckodriver, code opens Firefox (needs to be preinstalled) and 
#10x, it will scroll down to the end of the page, wait 4 seconds and then scroll to the end again

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import time

class Sel():
    def __init__(self):
        self.driver = webdriver.Firefox()
        self.driver.implicitly_wait(30)
        self.base_url = "https://twitter.com"
        self.verificationErrors = []
        self.accept_next_alert = True

        driver = self.driver
        delay = 3
        driver.get(self.base_url + "/search?q=merkel&src=typed_query")
        
        #driver.find_element_by_link_text("Alle anzeigen").click()
        for i in range(1,10):
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)
        html_source = driver.page_source
        data = html_source.encode('utf-8')
        print(html_source)

Sel()

# Using Selenium for scraping of websites that need a login

In [ ]:
#example use Selenium for managing the session and bs4 for scraping the data
#Selenium also offers the scraping functionalities but they are easier to use in bs4
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re

In [ ]:
driver = webdriver.Firefox()
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

In [ ]:
#find how the elements are called by checking the website (right-click on button -> Inspect)
#there are different ways to identify elements (by_id; by_name; by_xpath)
driver.find_element_by_id("username").send_keys("tempwebmining@tempr.email")
driver.find_element_by_name("session_password").send_keys("webmining")
driver.find_element_by_xpath("//button[@data-litms-control-urn='login-submit']").click()

In [ ]:
x=driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20science")
html_source = driver.page_source
data = html_source.encode('utf-8')

In [ ]:
soup = BeautifulSoup(data, 'html')
events = soup.findAll('div',{'class':re.compile("job-card-container*")})
print(len(events))

In [ ]:
for e in events:
    e1 = e.find('a',{'class':re.compile("^job-card-list__title.*")})
    if e1:
        print(e1.get_text())
    

We need to scroll down on the internal div to get all data

In [ ]:
#above code was scrolling down the entire page, however, we wanted to only scroll down the the job offering element
x=driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20science&start=0")
driver.execute_script("document.getElementsByClassName('jobs-search-results')[0].scrollTop=2000")
time.sleep(10)
html_source = driver.page_source
data = html_source


In [ ]:
soup = BeautifulSoup(data, 'html')
events = soup.findAll('li',{'class':re.compile("jobs-search-results__list-item*")})
print(len(events))

In [ ]:
for e in events:
    e1 = e.find('a',{'class':re.compile("^job-card-list__title.*")})
    if e1:
        print(e1.get_text())

## Skeleton code without Selenium (doesn't work with Linkedin)

In [ ]:
from requests import Session
from bs4 import BeautifulSoup

s=Session()
site = s.get("https://www.linkedin.com/login")
bs_content = bs(site.content, "html.parser")
loginCsrfParam = bs_content.find("input", {"name":"loginCsrfParam"})["value"]
csrfToken = bs_content.find("input", {"name": "csrfToken"})['value']
sIdString = bs_content.find("input", {"name": "sIdString"})['value']
print(loginCsrfParam)
print(csrfToken)
print(sIdString)


login_data = {"trk": "guest_homepage-basic_sign-in-submit","session_key":"tempwebmining@tempr.email","session_password":"webmining", "loginCsrfParam":loginCsrfParam, 'csrfToken': csrfToken,'sIdString': sIdString}
s.post("https://www.linkedin.com/checkpoint/lg/login-submit",login_data)
r = s.get("https://www.linkedin.com/jobs/search/?keywords=data%20science")
